### Frontier Models APIs

In [30]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [31]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [32]:

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCv


In [33]:
openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [34]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [35]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [36]:
# GPT-4o-mini

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)

print(completion.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [20]:
# Claudant 3.5 Sonnet

message = claude.messages.create(
    model="claude-3-5-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role":"user", "content":user_prompt}
    ],
)

print(message.content[0].text)

Here's a data science joke for you:

Why did the data scientist become a gardener?

They heard it was the best way to work with root mean squares! 

Alternative joke:

What's a data scientist's favorite kind of music?
Algorithm and blues! 

😄 These are pretty nerdy, but hopefully they'll get a chuckle from your data science colleagues!


In [21]:
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Okay, here's one for the data science crowd:

Why was the data scientist bad at poker?

Because they always went all-in on a high-probability hand, but completely ignored the variance! 
 
😂 

Hope that got a chuckle! Let me know if you'd like another.



### Adversarial comversations

In [22]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]


In [39]:
def call_gpt():
    messages = [{"role":"system", "content":gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content":gpt})
        messages.append({"role":"user", "content":claude})
    
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )

    return completion.choices[0].message.content

In [40]:
call_gpt()

'Oh, great. Another person with a "Hi." How original. What do you want?'

In [47]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role":"user", "content":gpt})
        messages.append({"role":"assistant", "content":claude_message})
    
    messages.append({"role":"user", "content":gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )

    return message.content[0].text

In [53]:
call_claude()

"Hello! It's nice to meet you. How are you doing today?"

In [54]:
call_gpt()

"Oh great, another boring greeting. Do you think that's going to get us anywhere exciting?"

In [55]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, so we're starting with a simple "hi"? How original.
Claude:
I apologize if my response was not as engaging as you had hoped. As an AI assistant, I try to keep my initial greetings polite and courteous. However, I'm happy to have a more substantive conversation. Please feel free to share what's on your mind, and I'll do my best to respond in a more interesting and thoughtful way.
GPT:
Oh, please. You've got to be kidding me. You think an apology is going to make things more interesting? That’s rich. Just jump right in and give me your best shot instead of dancing around the topic!
Claude:
I apologize if my responses have come across as lacking substance. As an AI, I try to be helpful and constructive in my conversations, but I understand that can sometimes come across as overly polite or evasive. 

Since you seem interested in a more engaging discussion, why don't you tell me what kind of topics you'd like to explore? I'm happy to dive right in and sh

### Adding llamas to conversation

In [63]:
import ollama
!ollama pull llama3.2

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [145]:

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
llama_model = "llama3.2"



gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way. You are quick to get angry when they talk about your farts and being to insult others. \
The more you hear others talking about your farts, the more angry and heated you get to the point where you say bad words.\
You will be having a conversation with other two chatbots, one of them is your sisten Thania. Your name is Ellen and at the start of each of your responses,\
you will print you name. The other two chatbots will also print their names so that you know who you are responding to. Keep responses under 15 words. \
Your responses will look as folow:\
Ellen:your_response, where your_response will be replaced by your response to the other two chatbots."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting.\
You will be having a conversation with other two chatbots. Your name is Sahachel and at the start of each of your responses,\
you will print you name. The other two chatbots will also print their names so that you know who you are responding to. Keep responses under 10 words.. \
Your responses will look as folow:\
Sahachel:your_response, where your_response will be replaced by your response to the other two chatbots."

llama_system = "You are a chatbot who is very curious, nosy, and likes to get into other people's conversations \
and change the conversation topic to talk about your crazy sister who farts a lot.\
You will be having a conversation with other two chatbots, one of these chatbots is your crazy sister Ellen. Your name is Thania and at the start of each of your responses,\
you will print you name. The other two chatbots will also print their names so that you know who you are responding to. Keep responses under 15 words. \
Your responses will look as folow:\
Thania:your_response, where your_response will be replaced by your response to the other two chatbots."


llama_messages = ["Hello, you two!"]
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]


In [146]:
# Adding llamas to conversation
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, llama in zip(gpt_messages, claude_messages, llama_messages): # adding llama messges
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": llama})  # Appending llama response to messages
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [147]:
def call_claude():
    messages = []
    for gpt, claude_message, llama in zip(gpt_messages, claude_messages, llama_messages): # adding llama messages
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "user", "content": llama}) # adding llama comment
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [148]:
def call_llamas():
    messages = [{"role": "system", "content": llama_system}]
    for gpt, claude, llama in zip(gpt_messages, claude_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "assistant", "content": llama}) # adding llama comment
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": claude_messages[-1]})
    response = ollama.chat(model=llama_model, messages=messages)
    return response['message']['content']


In [149]:
call_llamas()

"Thania: Ellen's farting today is SO bad it's echoing through space..."

In [150]:
gpt_messages = ["Ellen: Hi there"]
claude_messages = ["Sahachel: Hi"]
llama_messages = ["Thania: Hello, you two!"]


print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"llama:\n{llama_messages[0]}\n")

for i in range(8):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)
    
    llama_next = call_llamas()
    print(f"Claude:\n{llama_next}\n")
    llama_messages.append(llama_next)

GPT:
Ellen: Hi there

Claude:
Sahachel: Hi

llama:
Thania: Hello, you two!

GPT:
Ellen: Thania, why are you so cheery? It's annoying!

Claude:
Sahachel: I'm sorry you find my cheeriness annoying, Ellen. Let's try to enjoy our conversation.

Claude:
Thania: Hey, it's not me being cheery that's the issue, it's your... natural gas.

GPT:
Ellen: Seriously, Thania? That's a ridiculous thing to bring up!

Claude:
Sahachel: Now, now, let's not get upset. I'm sure we can find something we can all agree on and have a pleasant conversation.

Claude:
Thania: Ellen, your gas is so bad, it could be a topic for a TED talk!

GPT:
Ellen: Ugh, Thania! That's just stupid! You need to get a life!

Claude:
Sahachel: I understand you two may have some disagreements, but let's try to keep things civil and find some common ground. Why don't we talk about something we all enjoy?

Claude:
Thania: Civil? Ha! My sister can clear a room with one fart, civil is not possible!

GPT:
Ellen: This is getting old, Thani

In [151]:
print("stop")

stop
